## YOLO model training

In [ ]:
from ultralytics import YOLO

In [ ]:
!pip3 install torch torchvision torchaudio

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
torch.__version__

In [ ]:
import clearml
clearml.browser_login()

In [ ]:
#login with clearml credentials

%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
#get these two once you make an account on clearml
%env CLEARML_API_ACCESS_KEY=
%env CLEARML_API_SECRET_KEY=

In [ ]:
model = YOLO("Pipeline_models/Best_version_2.pt")

In [ ]:
from clearml import Task
task = Task.init(project_name="project", task_name="my_task")
results = model.train(data="/home/naveen/workspace/voxel/Pipline_workspace/dataset/dataset.yaml",epochs=100,batch=2,imgsz=1280,augment=True,save=True,save_period=1)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
from ultralytics import YOLO

model = YOLO("Pipeline_models/Best_version_2.pt")
model.tune(data="/home/naveen/workspace/voxel/Pipline_workspace/dataset/dataset.yaml", epochs=10, iterations=300, optimizer='AdamW', plots=False,batch=2, save=False, val=False)

In [ ]:
!yolo task=detect mode=predict model=Pipeline_models/best_version_3.pt conf=0.25 source='/home/naveen/workspace/voxel/Pipline_workspace/dataset/test/images' save=True

## SAHI Integration

In [ ]:
import cv2
import fiftyone as fo
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict

dataset = fo.load_dataset("pipeline_data")

In [ ]:
yolov8_model_path = "Pipeline_models/Best_version_2.pt"

In [ ]:
sahi_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=yolov8_model_path,
    confidence_threshold=0.3,
    device="cuda:0",
)

In [ ]:

for sample in dataset:
    
    result = get_sliced_prediction(
        sample.filepath,
        sahi_model,
        slice_height = 1280,
        slice_width = 1280,
        overlap_height_ratio = 0.2,
        overlap_width_ratio = 0.2
    )

    fiftyone_detections = result.to_fiftyone_detections()
    
    sample["sahi_predictions_v2_1280x1280"] = fo.Detections(detections=fiftyone_detections)
    sample.save()
  



## Adding data to fiftyone

In [ ]:
import fiftyone as fo
dataset_list = fo.list_datasets()
print(dataset_list)

In [ ]:
import os
import fiftyone as fo

list_of_images = []

for root, dirs, files in os.walk('data', topdown=True):
    for file in files:
        if file.endswith('.JPG'):
            list_of_images.append(os.path.join(root, file))

dataset = fo.Dataset.from_images(list_of_images, name = "pipeline_data")
dataset.persistent = True
dataset.save()

In [ ]:
from PIL import Image


def get_image_dimensions(image_path):
    try:
        with Image.open(image_path) as img:
            width, height = img.size
        return width, height
    except Exception as e:
        raise ValueError(f"Unable to read image dimensions for {image_path}: {str(e)}")

with fo.ProgressBar() as pb:
    for sample in pb(dataset):
        image_path = sample['filepath']
        
        try:
            iw, ih = get_image_dimensions(image_path)
        except ValueError as ve:
            print(ve)
            continue  

        detections = []
        text_file = sample['filepath'].replace('images', 'kitti_data')
        text_file = text_file.replace('.JPG', '.txt')
        

        with open(text_file, 'r') as fr:
            lines = fr.readlines()

            for line in lines:
                line = line.replace("ELECTRIC POLES","ELECTRIC_POLES")
                line = line.replace("PERMANENT STRUCTURES","PERMENANT_STRUCTURES")
                line = line.replace("PERMENANT STRUCTURES","PERMENANT_STRUCTURES")
                line = line.replace("TEMPORARY STRUCTURES","TEMPORARY_STRUCTURES")
                cls,_,_,_,x1,y1,x2,y2,_,_,_,_,_,_,_,score =  line.split(' ')
                x1,y1,x2,y2,score = float(x1),float(y1),float(x2),float(y2),float(score) 
                rel_box = [float(x1 / iw),float( y1 / ih),float( (x2-x1) / iw),float( (y2-y1) / ih)]
                detections.append(
                    fo.Detection(
                        label=cls,
                        bounding_box=rel_box,
                        confidence=score
                    )
                )
            
            
            sample["ground_truth"] = fo.Detections(detections=detections)
            sample.save()

In [ ]:
dataset

### Launching the fiftyone interface

In [ ]:
import fiftyone as fo
dataset = fo.load_dataset("pipeline_data")

In [ ]:
session = fo.launch_app(dataset)

### Adding metadata

In [ ]:
for sample in dataset:
    sample.metadata = None
    sample.save()

In [ ]:
import cv2
# from fiftyone import fo
from fiftyone.core.metadata import ImageMetadata
import os

# Load your existing dataset
# dataset = fo.load_dataset("/path/to/your/dataset")

# Iterate over each sample in the dataset and update metadata
for sample in dataset:
    image_path = sample.filepath

    # Read image using OpenCV
    img = cv2.imread(image_path)
    height, width, _ = img.shape

    # Update metadata
    sample.metadata = ImageMetadata(width=width, height=height)
    print(sample.filepath)
    sample.save()
# Save the updated dataset


### Merging detections and performing nms operation

In [ ]:
for sample in dataset:
    sample["merged_detections"] = sample.sahi_predictions_v2_1280x1280.copy()
    sample.merged_detections.merge(
        sample.sahi_predictions_v3_640x640,
        merge_lists=True, 
        overwrite=True,    
    )

    sample.save()

In [ ]:
import fiftyone.utils.labels as fol


fol.perform_nms(sample_collection=dataset, in_field="merged_detections", out_field="merged_nms", confidence_thresh=0.15, iou_thresh=0.10,classwise=True, progress=True)